# Google Setup

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

google_project_name = "msd8654-498-dev"
google_dataset_name = "usgs"

Authenticated


# Get data from usgs.gov

In [ ]:
import requests
import json
import pandas as pd
from google.cloud import bigquery
import urllib.request
import os

for state_cd in (['nj']):
  f = open("groundwater_sites_" + state_cd + ".tmp", 'w')
  f2 = open("groundwater_sites_" + state_cd + ".tsv", 'w')
  url = "https://nwis.waterdata.usgs.gov/nwis/gwlevels?state_cd=nj&group_key=NONE&format=sitefile_output&sitefile_output_format=rdb" \
  + "&column_name=agency_cd&column_name=site_no&column_name=station_nm&column_name=site_tp_cd&column_name=lat_va&column_name=long_va" \
  + "&column_name=dec_lat_va&column_name=dec_long_va&column_name=coord_meth_cd&column_name=coord_acy_cd&column_name=coord_datum_cd" \
  + "&column_name=dec_coord_datum_cd&column_name=district_cd&column_name=state_cd&column_name=county_cd&column_name=country_cd" \
  + "&column_name=land_net_ds&column_name=map_nm&column_name=map_scale_fc&column_name=alt_va&column_name=alt_meth_cd&column_name=alt_acy_va" \
  + "&column_name=alt_datum_cd&column_name=huc_cd&column_name=basin_cd&column_name=topo_cd&column_name=data_types_cd&column_name=instruments_cd" \
  + "&column_name=construction_dt&column_name=inventory_dt&column_name=drain_area_va&column_name=contrib_drain_area_va&column_name=tz_cd" \
  + "&column_name=local_time_fg&column_name=reliability_cd&column_name=gw_file_cd&column_name=nat_aqfr_cd&column_name=aqfr_cd&column_name=aqfr_type_cd" \
  + "&column_name=well_depth_va&column_name=hole_depth_va&column_name=depth_src_cd&column_name=project_no&column_name=rt_bol&column_name=peak_begin_date" \
  + "&column_name=peak_end_date&column_name=peak_count_nu&column_name=qw_begin_date&column_name=qw_end_date&column_name=qw_count_nu" \
  + "&column_name=gw_begin_date&column_name=gw_end_date&column_name=gw_count_nu&column_name=sv_begin_date" \
  + "&column_name=sv_end_date&column_name=sv_count_nu&date_format=YYYY-MM-DD&rdb_compression=file&list_of_search_criteria=state_cd"
  payload={}
  headers = {}
  response = requests.request("GET", url, headers=headers, data=payload)
  print("Response code:", response.status_code) 
  if (response.status_code>229):
    print("ERROR")
    break;
  else:
    f.writelines(response.text)
    f.close()

  for line in open("groundwater_sites_" + state_cd + ".tmp", 'r'):
    if (line.startswith("#") or line.startswith("5s")):
      #print(line)
      None
    else:
      f2.writelines(line)
  f2.close()

Response code: 200


# Upload file to GCS Bucket

In [ ]:
from google.cloud import storage
from google.cloud import bigquery
import urllib.request
import os

google_project_name = "msd8654-498-dev"
google_dataset_name = "usgs"

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the google storage bucket."""

    storage_client = storage.Client(project=google_project_name)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to Storage Bucket with Bob name  {} successfully .".format(
            source_file_name, destination_blob_name
        )
    )

# if running locally will need app credentials 
# terminal
# GOOGLE_APPLICATION_CREDENTIALS=<path>/<filename>.json
# powershell:
# $env:GOOGLE_APPLICATION_CREDENTIALS="<path>/<filename>.json"

upload_blob(google_project_name + "-" + google_dataset_name, './groundwater_sites_nj.tmp' ,'groundwater_sites_nj.tmp')



File ./groundwater_sites_nj.tmp uploaded to Storage Bucket with Bob name  groundwater_sites_nj.tmp successfully .


# Upload file to BigQuery Table

In [ ]:
# https://cloud.google.com/bigquery/docs/batch-loading-data#bq
import os
from google.cloud import bigquery

command = "bq load --source_format=CSV --field_delimiter=tab --autodetect=True " \
    + google_project_name + ":" + google_dataset_name + "." \
    + "groundwater_sites_nj" \
    + " ./groundwater_sites_nj.tsv"

file_path = "./groundwater_sites_nj.tsv"

# Construct a BigQuery client object.
client = bigquery.Client(project=google_project_name)

# TODO(developer): Set table_id to the ID of the table to create.
table_id =  google_dataset_name + "." + "groundwater_sites_nj" \

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV, 
    fieldDelimiter="\t",
    skip_leading_rows=1, 
    autodetect=True
)

with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

job.result()  # Waits for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 7002 rows and 56 columns to usgs.groundwater_sites_nj
